In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [3]:
problem_html = requests.get("https://atcoder.jp/contests/abc193/tasks/abc193_b")
problem_parse = BeautifulSoup(problem_html.content,"html.parser",from_encoding="utf-8")

In [16]:
problem_p = problem_parse.find_all("p")

In [17]:
[e.text for e in problem_p]

['Caddi Programming Contest 2021(AtCoder Beginner Contest 193) has begun.',
 'Caddi Programming Contest 2021(AtCoder Beginner Contest 193) has ended.',
 '\n\t\t\tTime Limit: 2 sec / Memory Limit: 1024 MB\n\t\t\t\n\t\t',
 '配点 : 200 点',
 '高橋くんは人気ゲーム機「スヌケマシン」を買おうとしています。\r\nスヌケマシンを販売している店は店 1, 2, \\dots, N の N 軒あり、店 i は高橋くんの現在地から徒歩 A_i 分、スヌケマシンの販売価格は P_i 円、現在のスヌケマシンの在庫は X_i 台です。\r\n高橋くんは今から徒歩でスヌケマシンを販売している店に向かい、店に着いたときにスヌケマシンの在庫があればスヌケマシンを買います。\r\nしかし、スヌケマシンは人気商品なので、今から 0.5, 1.5, 2.5, \\dots 分後に全ての店でスヌケマシンの在庫が (存在するなら) 1 台減ります。\r\n高橋くんがスヌケマシンを買うことができるか判定し、できる場合は買うのに必要な最小の金額を求めてください。',
 '入力は以下の形式で標準入力から与えられる。',
 '高橋くんがスヌケマシンを買うことができる場合は、買うのに必要な最小の金額を出力せよ。\r\nできない場合は、-1 を出力せよ。',
 '店 1 に向かうと、高橋くんが着いたときにはスヌケマシンが 2 台残っていて、9 円でスヌケマシンを買うことができます。\r\n店 2 に向かうと、高橋くんが着いたときにはスヌケマシンが 1 台残っていて、8 円でスヌケマシンを買うことができます。\r\n店 3 に向かうと、高橋くんが着いたときにはスヌケマシンが売り切れていて、買うことができません。  ',
 'Score : 200 points',
 "Takahashi wants to buy the popular video game console called Play Snuke.\r\nThere are N shops that sell Play S

In [2]:
def extract_text_from_atcoder_problem_page(problem_url):
    problem_html = requests.get(problem_url)
    problem_parse = BeautifulSoup(problem_html.content,"html.parser",from_encoding="utf-8")
    problem_div = problem_parse.find_all("div")
    #問題文の抽出,コドフォのページのhtmlをよく見て該当部分の属性で抽出
    res={"problem_text":[],"input_text":[],"output_text":[],"tags":[]}
    for elem in problem_div:
        try:
            elem_cls = elem.get("class")[0]
            if(elem_cls == "input-specification"):
                res["input_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])
            if(elem_cls == "output-specification"):
                res["output_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])
                break
        except:
            if(not res["problem_text"]):
                res["problem_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])

    res["problem_text"]="".join(res["problem_text"])
    res["input_text"]="".join(res["input_text"])
    res["output_text"]="".join(res["output_text"])
    #タグの抽出,コドフォのhtmlをよく見て該当部分の属性で抽出
    problem_span = problem_parse.find_all("span")
    for elem in problem_span:
        try:
            elem_cls = elem.get("class")[0]
            if(elem_cls == "tag-box"):
                res["tags"].append(elem.string)
        except:
            pass
    res["tags"]="".join(res["tags"])
    #assert type(res["problem_text"])==type(res["input_text"])==type(res["output_text"])==type(res["tags"])==str, "output is not strings"
    return res

In [29]:
base_url = "https://codeforces.com/problemset/problem/"
prob_nums = [str(i) for i in range(200,1500)] #200回から最新まで抽出
capital = [chr(i) for i in range(65, 65+9)] #ABCDEFGHI
prob_names = capital + [e+"1" for e in capital] #easyとhardがある場合があるがhardは割愛(問題文がほぼ同じため)

In [30]:
df = {"prob_id":[],"problem_texts":[],"input_texts":[],"output_texts":[],"tag_set":[]}

In [33]:
save_point=0
for num in prob_nums:
    for name in prob_names:
        prob_id = num+"/"+name
        try:
            datum = extract_text_from_cf_problem_page(base_url+prob_id)
            df["prob_id"].append(prob_id)
            df["problem_texts"].append(datum["problem_text"])
            df["input_texts"].append(datum["input_text"])
            df["output_texts"].append(datum["output_text"])
            df["tag_set"].append(datum["tags"])
        except:
            pass
        assert len(df["prob_id"])==len(df["problem_texts"])==len(df["input_texts"])==len(df["output_texts"])==len(df["tag_set"]), "does not the same length"
    if(save_point%10==0):
        pd.DataFrame(df).to_csv("../data/cf_problem_tag_dataset.csv",index=False)
        save_point+=1

In [27]:
pd.DataFrame(df).to_csv("../data/cf_problem_tag_dataset.csv",index=False)